In [ ]:
# Install necessary libraries
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.3.1 transformers==4.41.3 datasets==2.20.0 accelerate==0.31.0 --progress-bar off
!pip install -qqq bitsandbytes==0.43.1 peft==0.11.1 trl==0.9.4 openpyxl --progress-bar off  # For CSV and xlsx support


ERROR: Ignored the following yanked versions: 4.14.0, 4.25.0, 4.46.0
ERROR: Could not find a version that satisfies the requirement transformers==4.41.3 (from versions: 0.1, 2.0.0, 2.1.0, 2.1.1, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.8.0, 2.9.0, 2.9.1, 2.10.0, 2.11.0, 3.0.0, 3.0.1, 3.0.2, 3.1.0, 3.2.0, 3.3.0, 3.3.1, 3.4.0, 3.5.0, 3.5.1, 4.0.0rc1, 4.0.0, 4.0.1, 4.1.0, 4.1.1, 4.2.0, 4.2.1, 4.2.2, 4.3.0rc1, 4.3.0, 4.3.1, 4.3.2, 4.3.3, 4.4.0, 4.4.1, 4.4.2, 4.5.0, 4.5.1, 4.6.0, 4.6.1, 4.7.0, 4.8.0, 4.8.1, 4.8.2, 4.9.0, 4.9.1, 4.9.2, 4.10.0, 4.10.1, 4.10.2, 4.10.3, 4.11.0, 4.11.1, 4.11.2, 4.11.3, 4.12.0, 4.12.1, 4.12.2, 4.12.3, 4.12.4, 4.12.5, 4.13.0, 4.14.1, 4.15.0, 4.16.0, 4.16.1, 4.16.2, 4.17.0, 4.18.0, 4.19.0, 4.19.1, 4.19.2, 4.19.3, 4.19.4, 4.20.0, 4.20.1, 4.21.0, 4.21.1, 4.21.2, 4.21.3, 4.22.0, 4.22.1, 4.22.2, 4.23.0, 4.23.1, 4.24.0, 4.25.1, 4.26.0, 4.26.1, 4.27.0, 4.27.1, 4.27.2, 4.27.3, 4.27.4, 4.28.0, 4.28.1, 4.29.0, 4.29.1, 4.29.2, 4.30.0, 4.30.1, 4

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")



Using device: cuda


In [ ]:
import pandas as pd
# from google.colab import files

# Upload the questionanswer.csv file
# uploaded = files.upload()  # Select the questionanswer.csv file

# Load the dataset and check columns
df = pd.read_csv("questionanswer.csv", encoding='ISO-8859-1')
print(df.head())

# Ensure columns are as expected
assert set(df.columns) == {"Question", "Answer", "Type"}, "Columns must be: Question, Answer, Type"


                                            Question  \
0  About how many years ago did life originate on...   
1                        Example for Endemic species   
2            write a short notes lectron microscopes   
3         Explain the fine structure of mitochondria   
4  Indicate the major function of both vitamin K ...   

                                              Answer       Type  
0                                           3.5X10^9  Structure  
1  Dipterocurpus zeylanicus / Gracinia quaesita\n...  Structure  
2  Electron microscopes represent a groundbreakin...      Essay  
3  Fine Structure of Mitochondria\n\nMitochondria...      Essay  
4                                Aids blood clotting  Structure  


In [ ]:
from huggingface_hub import login

# Replace 'your_hf_token' with the actual token
login(token="")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"
PAD_TOKEN = "<|pad|>"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load tokenizer with padding
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, use_auth_token=True)
tokenizer.add_special_tokens({"pad_token": PAD_TOKEN})
tokenizer.padding_side = "right"

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto",
    use_auth_token=True
)
model.resize_token_embeddings(len(tokenizer))


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(128257, 4096)

In [ ]:
# Create formatted text with Type, Question, and Answer
def format_example(row):
    return f"Type: {row['Type']}\nQuestion: {row['Question']}\nAnswer: {row['Answer']}"

df['formatted'] = df.apply(format_example, axis=1)
print(df['formatted'].head())  # Verify formatting


0    Type: Structure\nQuestion: About how many year...
1    Type: Structure\nQuestion: Example for Endemic...
2    Type: Essay\nQuestion: write a short notes lec...
3    Type: Essay\nQuestion: Explain the fine struct...
4    Type: Structure\nQuestion: Indicate the major ...
Name: formatted, dtype: object


In [ ]:
# Convert DataFrame into Hugging Face Dataset
from datasets import Dataset

def format_example(row):
    return f"Type: {row['Type']}\nQuestion: {row['Question']}\nAnswer: {row['Answer']}"

df['formatted'] = df.apply(format_example, axis=1)

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['formatted']].rename(columns={"formatted": "text"}))

# Tokenize Dataset
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Split dataset
train_test = tokenized_dataset.train_test_split(test_size=0.2)
train_data = train_test["train"]
val_data = train_test["test"]


Map:   0%|          | 0/2354 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Configure LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)

# Prepare model for LoRA training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

from trl import DataCollatorForCompletionOnlyLM, SFTConfig, SFTTrainer

# Define a response template to pass into DataCollatorForCompletionOnlyLM
response_template = "<|endoftext|>"

# Set up Data Collator with the response template
data_collator = DataCollatorForCompletionOnlyLM(
    tokenizer=tokenizer,
    response_template=response_template,
    return_tensors="pt"
)

# Trainer configuration with minimal logging by setting logging_steps to a high value
sft_config = SFTConfig(
    output_dir="./llama_custom_model",
    max_seq_length=512,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    save_steps=100,  # Save checkpoints only
    logging_steps=1000000,  # Set to a high number to avoid frequent logging
    evaluation_strategy="no",  # Skip evaluation during training
    learning_rate=1e-4,
    fp16=True,
    report_to="none"
)

# Initialize Trainer without tracking training loss
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Start training without loss tracking or logging
trainer.train()
trainer.save_model("./llama_custom_model")
print("Model training completed and saved.")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:402: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:187: UserWarning: Could not find response key `<|endoftext|>` in the following instance: <|begin_of_text|>Type: Structure
Question: Name the components of apoplastic path?
Answer: Cell wall 
Extracellular space 
Vessel elements 
Tracheids <|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|

Step,Training Loss


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:187: UserWarning: Could not find response key `<|endoftext|>` in the following instance: <|begin_of_text|>Type: Structure
Question: What is the animal phylum that shows circulatory system is reduced and closed without a heart and larval forms are bilaterally symmetrical?
Answer: Echinodermata<|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|>

Model training completed and saved.


In [ ]:
# Save the model locally
OUTPUT_DIR = "./llama_custom_model"
trainer.save_model(OUTPUT_DIR)
print("Model saved locally at:", OUTPUT_DIR)


Model saved locally at: ./llama_custom_model


In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define the output directory
OUTPUT_DIR = "./llama_custom_model"
GOOGLE_DRIVE_PATH = "/content/drive/My Drive/llama_custom_model"

# Save the model locally
trainer.save_model(OUTPUT_DIR)
print("Model saved locally at:", OUTPUT_DIR)

# Copy the saved model to Google Drive
shutil.copytree(OUTPUT_DIR, GOOGLE_DRIVE_PATH, dirs_exist_ok=True)
print(f"Model saved to Google Drive at: {GOOGLE_DRIVE_PATH}")


Mounted at /content/drive
Model saved locally at: ./llama_custom_model
Model saved to Google Drive at: /content/drive/My Drive/llama_custom_model


In [1]:
!pip install -U bitsandbytes
!pip install -U transformers
!pip install -U accelerate
!pip install -U peft
!pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 115.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstall

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import PeftModel

# Base model and tokenizer
MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"
PAD_TOKEN = "<|pad|>"

# Define BitsAndBytesConfig for 4-bit quantization with NF4 and bfloat16
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print(f"Loading tokenizer from {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
print("Tokenizer loaded successfully.")

print("Adding <|pad|> token to tokenizer vocabulary...")
tokenizer.add_special_tokens({"pad_token": PAD_TOKEN})
print("Special token <|pad|> added.")

print(f"Loading base model from {MODEL_NAME} with 4-bit quantization...")
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config, # Use 4-bit quantization config
    device_map="auto"                       # Automatically map to available devices
)
print("Base model loaded successfully and quantized to 4-bit.")
print("Model mapped to available devices.")


Loading tokenizer from meta-llama/Meta-Llama-3-8B-Instruct...


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Tokenizer loaded successfully.
Adding <|pad|> token to tokenizer vocabulary...
Special token <|pad|> added.
Loading base model from meta-llama/Meta-Llama-3-8B-Instruct with 4-bit quantization...


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Base model loaded successfully and quantized to 4-bit.
Model mapped to available devices.


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

MODEL_PATH = "./llama_custom_model"
print("Loading fine-tuned LoRA weights from './llama_custom_model'...")
fine_tuned_model = PeftModel.from_pretrained(base_model, MODEL_PATH)
print("Fine-tuned LoRA weights loaded and merged with the base model.")
print("Fine-tuned model is ready for inference.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading fine-tuned LoRA weights from './llama_custom_model'...
Fine-tuned LoRA weights loaded and merged with the base model.
Fine-tuned model is ready for inference.


In [10]:
import torch
from peft import PeftModel

def merge_and_unload(peft_model):
    # Ensure the model is in eval mode
    peft_model.eval()

    # Iterate over each module and merge LoRA weights
    for name, module in peft_model.named_modules():
        if hasattr(module, "merge"):
            module.merge()  # Merge LoRA weights

    # Return the underlying base model after merging
    return peft_model.base_model


In [12]:
# Merge LoRA weights
merged_model = merge_and_unload(fine_tuned_model)

/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:1048: UserWarning: All adapters are already merged, nothing to do.
  warnings.warn("All adapters are already merged, nothing to do.")


In [13]:
merged_model.save_pretrained("./merged_model")
tokenizer.save_pretrained("./merged_model")
print("Merged model saved locally.")

Merged model saved locally.


In [14]:
import os

def get_directory_size(directory):
    """
    Calculate the total size of a directory in bytes.
    """
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for file in filenames:
            filepath = os.path.join(dirpath, file)
            total_size += os.path.getsize(filepath)
    return total_size

# Specify the path to the directory where the merged model is saved
merged_model_dir = "./merged_model"

# Calculate the size in bytes
size_in_bytes = get_directory_size(merged_model_dir)

# Convert the size to human-readable format
size_in_mb = size_in_bytes / (1024 * 1024)  # Convert bytes to megabytes
print(f"Merged model size: {size_in_mb:.2f} MB")


Merged model size: 5791.23 MB


In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

# Specify the path to the saved merged model and tokenizer
MODEL_PATH = "./merged_model"

# Load the tokenizer
print("Loading tokenizer from merged model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=True)
print("Tokenizer loaded successfully.")

# Define BitsAndBytesConfig for 4-bit quantization with NF4 and bfloat16
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the model with 4-bit quantization
print("Loading merged model and applying 4-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=quantization_config

)
print("Merged model successfully quantized to 4-bit.")


QUANTIZED_MODEL_PATH = "./quantized_model_4bit"
print(f"Saving 4-bit quantized model to {QUANTIZED_MODEL_PATH}...")
model.save_pretrained(QUANTIZED_MODEL_PATH)
tokenizer.save_pretrained(QUANTIZED_MODEL_PATH)
print("4-bit quantized model saved successfully.")


Loading tokenizer from merged model...


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Tokenizer loaded successfully.
Loading merged model and applying 4-bit quantization...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at ./merged_model were not used when initializing LlamaForCausalLM: ['model.layers.0.self_attn.q_proj.base_layer.weight', 'model.layers.0.self_attn.q_proj.base_layer.weight.absmax', 'model.layers.0.self_attn.q_proj.base_layer.weight.quant_map', 'model.layers.0.self_attn.q_proj.base_layer.weight.quant_state.bitsandbytes__nf4', 'model.layers.0.self_attn.q_proj.lora_A.default.weight', 'model.layers.0.self_attn.q_proj.lora_B.default.weight', 'model.layers.0.self_attn.v_proj.base_layer.weight', 'model.layers.0.self_attn.v_proj.base_layer.weight.absmax', 'model.layers.0.self_attn.v_proj.base_layer.weight.quant_map', 'model.layers.0.self_attn.v_proj.base_layer.weight.quant_state.bitsandbytes__nf4', 'model.layers.0.self_attn.v_proj.lora_A.default.weight', 'model.layers.0.self_attn.v_proj.lora_B.default.weight', 'model.layers.1.self_attn.q_proj.base_layer.weight', 'model.layers.1.self_attn.q_proj.base_layer.weight.absmax', 'model.layers.1.self_attn.q_proj.ba

Merged model successfully quantized to 4-bit.
Saving 4-bit quantized model to ./quantized_model_4bit...
4-bit quantized model saved successfully.


In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Path to save the model in Google Drive
drive_path = "/content/drive/My Drive/merged_model"

# Save merged model and tokenizer to Google Drive
shutil.copytree("./merged_model", drive_path, dirs_exist_ok=True)
print(f"Merged model saved in Google Drive at: {drive_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Merged model saved in Google Drive at: /content/drive/My Drive/merged_model


In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Path to save the model in Google Drive
drive_path = "/content/drive/My Drive/merged_model-4bit"

# Save merged model and tokenizer to Google Drive
shutil.copytree("./quantized_model_4bit", drive_path, dirs_exist_ok=True)
print(f"Merged model saved in Google Drive at: {drive_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Merged model saved in Google Drive at: /content/drive/My Drive/merged_model-4bit


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Reload the merged model and tokenizer
model = AutoModelForCausalLM.from_pretrained("./merged_model")
tokenizer = AutoTokenizer.from_pretrained("./merged_model")


# Initialize the pipeline for text generation
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    return_full_text=False
)

# Define a sample prompt
sample_prompt = "Question: What is the structure of DNA?\nInformation:\n"
print("Prompt:", sample_prompt)

# Generate response
print("Generating response...")
output = pipe(sample_prompt)
print("Generated Answer:", output[0]["generated_text"])


The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

Prompt: Question: What is the structure of DNA?
Information:

Generated Answer: A. Double helix
B. Single helix
C. Triple helix
D. Quadruple helix

Answer: A. Double helix
Explanation: DNA (Deoxyribonucleic acid) is a double-stranded helix, where two complementary strands are twisted together. Each strand is composed of nucleotides, which are linked by hydrogen bonds between the base pairs. The sugar-phosphate backbone provides the structural framework for the DNA molecule. The double helix structure of DNA was first proposed by James Watson and Francis Crick in 1953, based on X-ray crystallography data collected by
